In [50]:
# load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")
tables <- list(
  Orders   = readr::read_csv("../../data/processed/Orders.csv"),
  Returns  = readr::read_csv("../../data/processed/Returns.csv"),
  People   = readr::read_csv("../../data/processed/People.csv")
)
tables <- apply_factors(tables)
orders <- tables$Orders 
returns <- tables$Returns
people <- tables$People

Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1173 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): order_id, market
lgl (1): returned

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 14 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): person, region

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the co

# General Product Performance

### Q1. Which products generate the highest/lowest total sales?

In [2]:
product_sales <- orders %>%
    group_by(product_name) %>%
    summarize(
        total_sales = sum(sales),
        n_orders = n()
    ) %>%
    arrange(desc(total_sales))

product_sales %>% slice_head(n=10)

product_name,total_sales,n_orders
<chr>,<dbl>,<int>
"Apple Smart Phone, Full Size",86935.78,51
"Cisco Smart Phone, Full Size",76441.53,38
"Motorola Smart Phone, Full Size",73156.30,38
"Nokia Smart Phone, Full Size",71904.56,47
Canon imageCLASS 2200 Advanced Copier,61599.82,5
"Hon Executive Leather Armchair, Adjustable",58193.48,49
"Office Star Executive Leather Armchair, Adjustable",50661.68,45
"Harbour Creations Executive Leather Armchair, Adjustable",50121.52,39
"Samsung Smart Phone, Cordless",48653.46,26


In [3]:
product_sales %>% slice_tail(n=10)

product_name,total_sales,n_orders
<chr>,<dbl>,<int>
Newell 308,8.400,2
Stockwell Gold Paper Clips,8.096,2
4009 Highlighters,8.040,1
Xerox 1989,7.968,1
"Avery Hi-Liter Comfort Grip Fluorescent Highlighter, Yellow Ink",7.800,2
Avery Hi-Liter Pen Style Six-Color Fluorescent Set,7.700,1
Grip Seal Envelopes,7.072,1
Xerox 20,6.480,1
Avery 5,5.760,1


### Q2. Which product categories perform best/worst?

In [4]:
category_sales <- orders %>%
    group_by(category) %>%
    summarize(
        total_sales = sum(sales),
        n_orders = n()
    ) %>%
    arrange(desc(total_sales))

category_sales

category,total_sales,n_orders
<fct>,<dbl>,<int>
Technology,4744557,10141
Furniture,4110874,9876
Office Supplies,3787070,31273


### Q3. Which product sub-categories perform best/worst?

In [5]:
subcategory_sales <- orders %>% 
    group_by(category, sub_category) %>% 
    summarize(
        total_sales = sum(sales),
        n_orders = n()
    ) %>% 
    arrange(desc(total_sales))

subcategory_sales

`summarise()` has grouped output by 'category'. You can override using the
`.groups` argument.


category,sub_category,total_sales,n_orders
<fct>,<fct>,<dbl>,<int>
Technology,Phones,1706824.14,3357
Technology,Copiers,1509436.27,2223
Furniture,Chairs,1501681.76,3434
Furniture,Bookcases,1466572.24,2411
Office Supplies,Storage,1127085.86,5059
Office Supplies,Appliances,1011064.30,1755
Technology,Machines,779060.07,1486
Furniture,Tables,757041.92,861
Technology,Accessories,749237.02,3075


# Revenue Structure & Concentration

### Q4. How concentrated are sales among top products/sub-categories?

In [6]:
product_sales <- product_sales %>%
    arrange(desc(total_sales)) %>%
    mutate(
        cumulative_sales = cumsum(total_sales),
        cumulative_share = cumulative_sales / sum(total_sales),
        product_rank = row_number(),
        product_share = product_rank / n()
    )

product_sales %>%
    select(product_rank, product_name, total_sales, cumulative_share) %>%
    slice_head(n = 20)

product_rank,product_name,total_sales,cumulative_share
<int>,<chr>,<dbl>,<dbl>
1,"Apple Smart Phone, Full Size",86935.78,0.006876469
2,"Cisco Smart Phone, Full Size",76441.53,0.012922862
3,"Motorola Smart Phone, Full Size",73156.30,0.018709399
4,"Nokia Smart Phone, Full Size",71904.56,0.024396925
5,Canon imageCLASS 2200 Advanced Copier,61599.82,0.029269364
6,"Hon Executive Leather Armchair, Adjustable",58193.48,0.033872368
7,"Office Star Executive Leather Armchair, Adjustable",50661.68,0.037879619
8,"Harbour Creations Executive Leather Armchair, Adjustable",50121.52,0.041844144
9,"Samsung Smart Phone, Cordless",48653.46,0.045692549


In [7]:
subcategory_sales <- subcategory_sales %>%
    ungroup() %>%
    arrange(desc(total_sales)) %>%
    mutate(
        cumulative_sales = cumsum(total_sales),
        cumulative_share = cumulative_sales / sum(total_sales),
        subcategory_rank = row_number(),
        subcategory_share = subcategory_rank / n()
    )

subcategory_sales %>% select(subcategory_rank, sub_category, total_sales, cumulative_share) 

subcategory_rank,sub_category,total_sales,cumulative_share
<int>,<fct>,<dbl>,<dbl>
1,Phones,1706824.14,0.1350068
2,Copiers,1509436.27,0.2544006
3,Chairs,1501681.76,0.3731811
4,Bookcases,1466572.24,0.4891844
5,Storage,1127085.86,0.5783349
6,Appliances,1011064.30,0.6583084
7,Machines,779060.07,0.7199307
8,Tables,757041.92,0.7798114
9,Accessories,749237.02,0.8390747


### Q5. What share of products generate little or no revenue?

In [20]:
product_sales <- product_sales %>% ungroup()
sales_threshold <- quantile(product_sales$total_sales, 0.1)

low_revenue_products <- product_sales %>%
  filter(total_sales <= sales_threshold)

low_revenue_summary <- product_sales %>%
    summarise(
      total_products = n(),
      low_revenue_products = sum(total_sales <= quantile(total_sales, 0.1)),
      share_low_revenue = low_revenue_products / total_products
    )

low_revenue_summary


total_products,low_revenue_products,share_low_revenue
<int>,<int>,<dbl>
3788,379,0.1000528


In [21]:
low_revenue_detailed <- orders %>%
    select(product_name, category, sub_category, sales) %>%
    inner_join(low_revenue_products, by = "product_name")


low_revenue_summary_cat <- low_revenue_detailed %>%
    group_by(category, sub_category) %>%
    summarise(
        n_products = n_distinct(product_name),
        total_sales = sum(sales),
        avg_sales_per_product = total_sales / n_products,
        share_of_low_revenue_sales =
        total_sales / sum(total_sales),
        .groups = "drop"
    ) %>%
    arrange(desc(total_sales))

low_revenue_summary_cat

category,sub_category,n_products,total_sales,avg_sales_per_product,share_of_low_revenue_sales
<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>
Office Supplies,Paper,81,4104.412,50.67175,1
Office Supplies,Art,84,3604.216,42.90733,1
Office Supplies,Binders,56,2731.034,48.76846,1
Office Supplies,Labels,34,1483.172,43.62271,1
Furniture,Furnishings,26,1384.336,53.24369,1
Office Supplies,Fasteners,25,962.964,38.51856,1
Office Supplies,Envelopes,15,804.702,53.64680,1
Technology,Accessories,14,580.708,41.47914,1
Technology,Phones,10,573.644,57.36440,1


# Volume vs Value

### Q6. Are top-selling products high-volume or high-value?

In [23]:
product_volume_value <- orders %>%
    group_by(product_name) %>%
    summarise(
        total_sales = sum(sales),
        total_quantity = sum(quantity),
        avg_price_per_unit = total_sales / total_quantity,
        n_orders = n(),
        .groups = "drop"
    )

top_products <- product_volume_value %>%
    arrange(desc(total_sales)) %>%
    slice_head(n = 20)

top_products

product_name,total_sales,total_quantity,avg_price_per_unit,n_orders
<chr>,<dbl>,<dbl>,<dbl>,<int>
"Apple Smart Phone, Full Size",86935.78,171,508.3964,51
"Cisco Smart Phone, Full Size",76441.53,139,549.9391,38
"Motorola Smart Phone, Full Size",73156.30,134,545.9426,38
"Nokia Smart Phone, Full Size",71904.56,147,489.1466,47
Canon imageCLASS 2200 Advanced Copier,61599.82,20,3079.9912,5
"Hon Executive Leather Armchair, Adjustable",58193.48,169,344.3401,49
"Office Star Executive Leather Armchair, Adjustable",50661.68,141,359.3027,45
"Harbour Creations Executive Leather Armchair, Adjustable",50121.52,142,352.9684,39
"Samsung Smart Phone, Cordless",48653.46,108,450.4950,26


### Q7. Which products have high order frequency but low total sales?

In [24]:
frequency_threshold <- quantile(product_volume_value$n_orders, 0.75)
sales_threshold <- quantile(product_volume_value$total_sales, 0.25)

high_freq_low_sales <- product_volume_value %>%
    filter(
        n_orders >= frequency_threshold,
        total_sales <= sales_threshold
    ) %>%
    arrange(desc(n_orders))

high_freq_low_sales

product_name,total_sales,total_quantity,avg_price_per_unit,n_orders
<chr>,<dbl>,<dbl>,<dbl>,<int>
"Novimex Round Labels, Adjustable",210.9464,67,3.148454,22
Avery Non-Stick Binders,217.3160,71,3.060789,20
"Avery Round Labels, Adjustable",258.0144,65,3.969452,20
"Hon Round Labels, Adjustable",250.7088,68,3.686894,20
"Novimex Round Labels, Alphabetical",250.5020,63,3.976222,19


# Category Structure

### Q8. How diverse is revenue within each category?

In [26]:
category_product_sales <- orders %>%
    group_by(category, product_name) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    )
category_diversity <- category_product_sales %>%
    group_by(category) %>% 
    summarise(
        n_products = n(),
        total_category_sales = sum(total_sales),
        top_product_share = max(total_sales) / sum(total_sales),
        sales_sd = sd(total_sales),
        sales_cv = sales_sd / mean(total_sales),
        .groups = "drop"
    ) %>%
    arrange(desc(top_product_share))

category_diversity

category,n_products,total_category_sales,top_product_share,sales_sd,sales_cv
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Technology,876,4744557,0.018323264,8145.836,1.503987
Furniture,842,4110874,0.014155988,6568.877,1.345454
Office Supplies,2071,3787070,0.009080299,3657.184,1.999970


### Q9. Are sub-categories performing consistently within categories?

In [27]:
subcategory_sales <- orders %>%
    group_by(category, sub_category) %>%
    summarise(
        total_sales = sum(sales),
        n_products = n_distinct(product_name),
        .groups = "drop"
    )
subcategory_consistency <- subcategory_sales %>%
    group_by(category) %>%
    summarise(
        n_subcategories = n(),
        mean_sales = mean(total_sales),
        sd_sales = sd(total_sales),
        cv_sales = sd_sales / mean_sales,
        max_to_min_ratio = max(total_sales) / min(total_sales),
        .groups = "drop"
    ) %>%
    arrange(desc(cv_sales))
subcategory_consistency

category,n_subcategories,mean_sales,sd_sales,cv_sales,max_to_min_ratio
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Office Supplies,9,420785.6,389090.8,0.9246770,15.354550
Furniture,4,1027718.5,548587.2,0.5337913,3.894623
Technology,4,1186139.4,494041.4,0.4165121,2.278083


# Stability & Consistency

### Q10. Are top products consistently strong across regions?

In [28]:
top_products <- product_sales %>%
    arrange(desc(total_sales)) %>%
    slice_head(n = 20) %>%
    pull(product_name)

product_region_sales <- orders %>%
    filter(product_name %in% top_products) %>%
    group_by(product_name, region) %>%
    summarise(
        regional_sales = sum(sales),
        .groups = "drop"
    )

product_region_consistency <- product_region_sales %>%
    group_by(product_name) %>%
    summarise(
        n_regions = n(),
        mean_sales = mean(regional_sales),
        sd_sales = sd(regional_sales),
        cv_sales = sd_sales / mean_sales,
        max_to_min_ratio = max(regional_sales) / min(regional_sales),
        .groups = "drop"
    ) %>%
    arrange(cv_sales)

product_region_consistency

product_name,n_regions,mean_sales,sd_sales,cv_sales,max_to_min_ratio
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Canon imageCLASS 2200 Advanced Copier,3,20533.275,8467.758,0.4123920,2.150000
"Harbour Creations Executive Leather Armchair, Adjustable",9,5569.057,2455.356,0.4408925,4.032839
"Motorola Smart Phone, Cordless",9,4325.671,2208.118,0.5104682,9.833333
"Hoover Stove, White",9,3649.178,1962.395,0.5377635,5.666667
"Cisco Smart Phone, Full Size",10,7644.153,4162.284,0.5445055,9.440052
"Apple Smart Phone, Full Size",10,8693.578,4766.375,0.5482639,5.701215
"Sauder Classic Bookcase, Traditional",10,3910.830,2451.227,0.6267793,13.846154
"Eldon File Cart, Single Width",9,3820.859,2420.586,0.6335189,33.369358
"Motorola Smart Phone, Full Size",11,6650.573,4358.910,0.6554187,18.215084


### Q11. Do products perform consistently across customer segments?

In [29]:
product_segment_sales <- orders %>%
    filter(product_name %in% top_products) %>%
    group_by(product_name, segment) %>%
    summarise(
        segment_sales = sum(sales),
        .groups = "drop"
    )

product_segment_consistency <- product_segment_sales %>%
    group_by(product_name) %>%
    summarise(
        n_segments = n(),
        mean_sales = mean(segment_sales),
        sd_sales = sd(segment_sales),
        cv_sales = sd_sales / mean_sales,
        max_to_min_ratio = max(segment_sales) / min(segment_sales),
        .groups = "drop"
    ) %>%
    arrange(cv_sales)

product_segment_consistency

product_name,n_segments,mean_sales,sd_sales,cv_sales,max_to_min_ratio
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
"Sauder Classic Bookcase, Traditional",3,13036.10,3244.376,0.2488763,1.671254
"Apple Smart Phone, Full Size",3,28978.59,8941.616,0.3085594,1.754631
"Office Star Executive Leather Armchair, Adjustable",3,16887.23,5988.987,0.3546460,1.839752
"Novimex Executive Leather Armchair, Adjustable",3,13528.38,4891.398,0.3615658,2.167203
"Hoover Stove, White",3,10947.53,4140.998,0.3782585,1.977825
"Motorola Smart Phone, Full Size",3,24385.43,10536.642,0.4320875,2.607857
"Motorola Smart Phone, Cordless",3,12977.01,5855.518,0.4512223,2.420000
"Eldon File Cart, Single Width",3,11462.58,5198.821,0.4535473,2.245288
Canon imageCLASS 2200 Advanced Copier,3,20533.27,11163.450,0.5436761,2.937500


# Underperformance Diagnostics

### Q12. Are underperforming products concentrated in specific categories/sub-categories?

In [36]:
sales_threshold <- quantile(product_sales$total_sales, 0.1)

underperforming_products <- product_sales %>%
    filter(total_sales <= sales_threshold)

underperforming_by_category <- orders %>%
    filter(product_name %in% underperforming_products$product_name) %>%
    group_by(category) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    )

underperforming_by_category

underperforming_by_subcategory <- orders %>%
    filter(product_name %in% underperforming_products$product_name) %>%
    group_by(category, sub_category) %>%
    summarise(
        total_sales = sum(sales),
        .groups = "drop"
    ) %>%
    arrange(desc(total_sales))
    
underperforming_by_subcategory

category,total_sales
<fct>,<dbl>
Furniture,1474.262
Office Supplies,14961.632
Technology,1298.158


category,sub_category,total_sales
<fct>,<fct>,<dbl>
Office Supplies,Paper,4104.412
Office Supplies,Art,3604.216
Office Supplies,Binders,2731.034
Office Supplies,Labels,1483.172
Furniture,Furnishings,1384.336
Office Supplies,Fasteners,962.964
Office Supplies,Envelopes,804.702
Technology,Accessories,580.708
Technology,Phones,573.644


### Q13. Are there products with high sales but unusually high return rates?

In [43]:
orders_with_returns <- orders %>%
    inner_join(returns, by = "order_id", relationship = "many-to-many") %>%
    select(order_id, product_name, category, sub_category, sales)

product_returns <- orders_with_returns %>%
    group_by(product_name) %>%
    summarise(
        total_returned_sales = sum(sales),
        n_returns = n(),
        .groups = "drop"
    ) %>%
    arrange(desc(total_returned_sales))

high_sales_threshold <- quantile(product_returns$total_returned_sales, 0.75)
high_return_threshold <- quantile(product_returns$n_returns, 0.75)

high_sales_high_returns <- product_returns %>%
    filter(
      total_returned_sales >= high_sales_threshold,
      n_returns >= high_return_threshold
    ) %>%
    arrange(desc(n_returns))

high_sales_high_returns


product_name,total_returned_sales,n_returns
<chr>,<dbl>,<int>
Staples,684.6100,17
"HP Copy Machine, Color",6980.3586,8
"Samsung Smart Phone, VoIP",17220.8960,6
"KitchenAid Refrigerator, White",5765.8368,5
"Samsung Audio Dock, Cordless",2613.0720,5
"Nokia Signal Booster, Cordless",2536.6345,5
"Hon Rocking Chair, Red",1369.3330,5
"Novimex Steel Folding Chair, Set of Two",1161.6000,5
"Office Star Steel Folding Chair, Red",1135.0641,5


# Lifecycle & Time

### Q14. Do products show declining or improving performance over time?

In [52]:
orders_with_period <- orders %>%
  add_time_period("order_date", period = "year")

top_products <- product_sales %>%
  arrange(desc(total_sales)) %>%
  slice_head(n = 10) %>%
  pull(product_name)

product_time_sales <- orders_with_period %>%
  filter(product_name %in% top_products) %>%
  group_by(product_name, period) %>%
  summarise(
    total_sales = sum(sales, na.rm = TRUE),
    .groups = "drop"
  )

product_time_sales

product_name,period,total_sales
<chr>,<date>,<dbl>
"Apple Smart Phone, Full Size",2011-01-01,14986.246
"Apple Smart Phone, Full Size",2012-01-01,19541.873
"Apple Smart Phone, Full Size",2013-01-01,21120.408
"Apple Smart Phone, Full Size",2014-01-01,31287.252
Canon imageCLASS 2200 Advanced Copier,2013-01-01,25899.926
Canon imageCLASS 2200 Advanced Copier,2014-01-01,35699.898
"Cisco Smart Phone, Full Size",2011-01-01,6662.650
"Cisco Smart Phone, Full Size",2012-01-01,11793.170
"Cisco Smart Phone, Full Size",2013-01-01,26737.133
